In [ ]:
#pip install supabase

In [ ]:
#pip install pandas

In [ ]:
#pip install python-dotenv

1. Supabase 셋업

In [1]:
import os
import dotenv
import pandas as pd
from supabase import create_client, Client

dotenv_file = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_file)

url: str = os.environ["SUPABASE_URL"]
key: str = os.environ["SUPABASE_KEY"]
supabase: Client = create_client(url, key)

2. 파일명과 테이블명 입력 및 데이터프레임 셋업

In [11]:
data_file = "routine.csv"
table_name = "ExerciseRoutine"
# 업데이트할 컬럼 리스트
update_columns = ['Name', 'ExList', 'MainGoal', 'DetailGoal', 'Delimiter', 'Level']

update_data = pd.read_csv(data_file)
response = supabase.table(table_name).select("*").execute()
origin_data = pd.DataFrame(response.data)

# 데이터 확인
print(update_data.shape)
print(origin_data.shape)
update_data.head()
origin_data.head()

(99, 9)
(100, 9)


,id,Name,Level,MainGoal,DetailGoal,ExList,MainImage,SubImage,Delimiter
0,999,테스트 루틴,Easy,다이어트,탄탄한 상체,"[1, 302, 25, 303]",https://nherivcnanphkalgybji.supabase.co/stora...,https://nherivcnanphkalgybji.supabase.co/stora...,전신
1,86,강력한 코어 만들기 중급,Normal,체력향상,코어 강화,"[901, 929, 930, 904, 302, 304, 305, 306, 307, ...",https://nherivcnanphkalgybji.supabase.co/stora...,https://nherivcnanphkalgybji.supabase.co/stora...,복근
2,10,복근 만들기 초급,Easy,근력강화,선명한 복근,"[901, 902, 904, 47, 48, 56, 72, 74, 47, 48, 56...",https://nherivcnanphkalgybji.supabase.co/stora...,https://nherivcnanphkalgybji.supabase.co/stora...,복근
3,59,상체근력 및 체력강화 중급,Normal,체력향상,균형 잡힌 라인,"[901, 909, 910, 42, 68, 55, 302, 45]",https://nherivcnanphkalgybji.supabase.co/stora...,https://nherivcnanphkalgybji.supabase.co/stora...,상체 근력 및 유산소
4,65,힙업과 군살 제거 중급,Normal,체력향상,균형 잡힌 라인,"[905, 931, 932, 909, 910, 25, 39, 40, 85, 35, ...",https://nherivcnanphkalgybji.supabase.co/stora...,https://nherivcnanphkalgybji.supabase.co/stora...,하체 근력 및 유산소


3. id 기준 수정할 컬럼에 대해 데이터프레임 병합(inner join) 후 update_data만 남김

In [12]:
# 공통된 id가 있는 row만 데이터프레임 병합
merged_df = origin_data.merge(update_data[['id']+update_columns], on='id', suffixes=('_orig','_new'))
# 업데이트 할 데이터 suffix '_new'를 다시 원래 컬럼 이름으로 변환
new_data = merged_df[['id']+[f"{col}_new" for col in update_columns]].rename(
    columns={f"{col}_new": col for col in update_columns}
) 
# Side 컬럼의 데이터 타입을 float에서 int로 변환
if 'Side' in update_columns:
    new_data['Side'] = new_data['Side'].astype(int)

# 데이터프레임을 딕셔너리 리스트로 변환
new_dict = new_data.to_dict(orient='records')
new_dict.__sizeof__()

904

In [13]:
new_dict

[{'id': 86,
  'Name': '강력한 코어 만들기 중급',
  'ExList': '[901,929,930,904,302,304,305,306,307,68,42,302] ',
  'MainGoal': '체력향상',
  'DetailGoal': '코어 강화',
  'Delimiter': '복근',
  'Level': 'Normal'},
 {'id': 10,
  'Name': '복근 만들기 초급',
  'ExList': '[901,902,904,47,48,56,72,74,47,48,56,72,74,301]',
  'MainGoal': '근력강화',
  'DetailGoal': '선명한 복근',
  'Delimiter': '복근',
  'Level': 'Easy'},
 {'id': 59,
  'Name': '상체근력 및 체력강화 중급',
  'ExList': '[901,909,910,42,68,55,302,45]',
  'MainGoal': '체력향상',
  'DetailGoal': '균형 잡힌 라인',
  'Delimiter': '상체 근력 및 유산소',
  'Level': 'Normal'},
 {'id': 65,
  'Name': '힙업과 군살 제거 중급',
  'ExList': '[905,931,932,909,910,25,39,40,85,35,42,28,64,25,39,40,85,35,42,28,64,25,39,40,85,35,42,28,64]',
  'MainGoal': '체력향상',
  'DetailGoal': '균형 잡힌 라인',
  'Delimiter': '하체 근력 및 유산소',
  'Level': 'Normal'},
 {'id': 85,
  'Name': '강력한 코어 만들기 초급',
  'ExList': '[901,929,930,904,302,59,51,49,46,304,305,302]',
  'MainGoal': '체력향상',
  'DetailGoal': '코어 강화',
  'Delimiter': '복근',
  'Level': 'Easy

In [14]:
# 공백제거
for item in new_dict:
    if item['id'] == 86:
        item['ExList'] = item['ExList'].strip()

In [15]:
new_dict

[{'id': 86,
  'Name': '강력한 코어 만들기 중급',
  'ExList': '[901,929,930,904,302,304,305,306,307,68,42,302]',
  'MainGoal': '체력향상',
  'DetailGoal': '코어 강화',
  'Delimiter': '복근',
  'Level': 'Normal'},
 {'id': 10,
  'Name': '복근 만들기 초급',
  'ExList': '[901,902,904,47,48,56,72,74,47,48,56,72,74,301]',
  'MainGoal': '근력강화',
  'DetailGoal': '선명한 복근',
  'Delimiter': '복근',
  'Level': 'Easy'},
 {'id': 59,
  'Name': '상체근력 및 체력강화 중급',
  'ExList': '[901,909,910,42,68,55,302,45]',
  'MainGoal': '체력향상',
  'DetailGoal': '균형 잡힌 라인',
  'Delimiter': '상체 근력 및 유산소',
  'Level': 'Normal'},
 {'id': 65,
  'Name': '힙업과 군살 제거 중급',
  'ExList': '[905,931,932,909,910,25,39,40,85,35,42,28,64,25,39,40,85,35,42,28,64,25,39,40,85,35,42,28,64]',
  'MainGoal': '체력향상',
  'DetailGoal': '균형 잡힌 라인',
  'Delimiter': '하체 근력 및 유산소',
  'Level': 'Normal'},
 {'id': 85,
  'Name': '강력한 코어 만들기 초급',
  'ExList': '[901,929,930,904,302,59,51,49,46,304,305,302]',
  'MainGoal': '체력향상',
  'DetailGoal': '코어 강화',
  'Delimiter': '복근',
  'Level': 'Easy'

In [16]:
# ExList 값을 PostgreSQL 배열 형식에 맞게 중괄호로 변환
for record in new_dict:
    if 'ExList' in record:
        record['ExList'] = record['ExList'].replace('[', '{').replace(']', '}').strip()

print(new_dict)

[{'id': 86, 'Name': '강력한 코어 만들기 중급', 'ExList': '{901,929,930,904,302,304,305,306,307,68,42,302}', 'MainGoal': '체력향상', 'DetailGoal': '코어 강화', 'Delimiter': '복근', 'Level': 'Normal'}, {'id': 10, 'Name': '복근 만들기 초급', 'ExList': '{901,902,904,47,48,56,72,74,47,48,56,72,74,301}', 'MainGoal': '근력강화', 'DetailGoal': '선명한 복근', 'Delimiter': '복근', 'Level': 'Easy'}, {'id': 59, 'Name': '상체근력 및 체력강화 중급', 'ExList': '{901,909,910,42,68,55,302,45}', 'MainGoal': '체력향상', 'DetailGoal': '균형 잡힌 라인', 'Delimiter': '상체 근력 및 유산소', 'Level': 'Normal'}, {'id': 65, 'Name': '힙업과 군살 제거 중급', 'ExList': '{905,931,932,909,910,25,39,40,85,35,42,28,64,25,39,40,85,35,42,28,64,25,39,40,85,35,42,28,64}', 'MainGoal': '체력향상', 'DetailGoal': '균형 잡힌 라인', 'Delimiter': '하체 근력 및 유산소', 'Level': 'Normal'}, {'id': 85, 'Name': '강력한 코어 만들기 초급', 'ExList': '{901,929,930,904,302,59,51,49,46,304,305,302}', 'MainGoal': '체력향상', 'DetailGoal': '코어 강화', 'Delimiter': '복근', 'Level': 'Easy'}, {'id': 81, 'Name': '건강한 몸매 만들기 상급', 'ExList': '{901,906,905,9

4. new_dict에서 하나씩 레코드를 가져와서 테이블에 업데이트

In [17]:
# Supabase Exercise 테이블에 update
for record in new_dict:
    # print(record)
    response_update = supabase.table(table_name).update(record).eq('id', record['id']).execute()
    # print(f"Update response for id {record['id']}:", response_update)
    print(f"Update response for id {record['id']}")

Update response for id 86
Update response for id 10
Update response for id 59
Update response for id 65
Update response for id 85
Update response for id 81
Update response for id 11
Update response for id 12
Update response for id 13
Update response for id 1
Update response for id 2
Update response for id 3
Update response for id 4
Update response for id 5
Update response for id 6
Update response for id 7
Update response for id 8
Update response for id 9
Update response for id 14
Update response for id 15
Update response for id 16
Update response for id 17
Update response for id 18
Update response for id 19
Update response for id 21
Update response for id 22
Update response for id 23
Update response for id 24
Update response for id 25
Update response for id 26
Update response for id 27
Update response for id 28
Update response for id 29
Update response for id 30
Update response for id 31
Update response for id 32
Update response for id 33
Update response for id 34
Update response for i